In [ ]:
import zipfile
import json
import polars as pl
import os

# CONFIGURACIÓN
ZIP_PATH = CARPETA_DATA = r"./Data/" + "esports_data.zip" # <--- Tu nuevo archivo ZIP
OUTPUT_META_FILE = "draft_oracle_tournament_meta.parquet"

def process_tournament_data():
    print("Analizando Meta de Torneos (Estructura Complex JSON)...")

    if not os.path.exists(ZIP_PATH):
        print(f"No encuentro {ZIP_PATH}. Súbelo primero.")
        return

    data_rows = []


    BAN_INDICES = {1, 2, 3, 4, 5, 6, 13, 14, 15, 16}

    with zipfile.ZipFile(ZIP_PATH, 'r') as z:

        json_files = [f for f in z.namelist() if f.endswith("series_details.json")]
        print(f"   Procesando {len(json_files)} series encontradas...")

        for json_file in json_files:
            try:
                with z.open(json_file) as f:
                    series_data = json.load(f)


                    if isinstance(series_data, dict): series_list = [series_data]
                    else: series_list = series_data

                    for series in series_list:
                        games = series.get("games", [])

                        for game in games:
                            teams = game.get("teams", [])
                            if len(teams) < 2: continue


                            winner_name = None
                            for team in teams:
                                objectives = team.get("objectives", [])

                                if any(obj.get("type") == "destroyNexus" for obj in objectives):
                                    winner_name = team.get("name")
                                    break


                            for team in teams:
                                team_name = team.get("name")
                                is_winner = (team_name == winner_name)

                                for player in team.get("players", []):
                                    char_info = player.get("character", {})
                                    c_name = char_info.get("name")

                                    if c_name:
                                        data_rows.append({
                                            "champion": c_name,
                                            "type": "PICK",
                                            "win": 1 if is_winner else 0
                                        })


                            actions = game.get("draftActions", [])
                            for action in actions:

                                try:
                                    seq = int(action.get("sequenceNumber", 0))
                                except: continue

                                if seq in BAN_INDICES:
                                    draftable = action.get("draftable", {})
                                    c_name = draftable.get("name")
                                    if c_name:
                                        data_rows.append({
                                            "champion": c_name,
                                            "type": "BAN",
                                            "win": 0
                                        })

            except Exception as e:
                continue


    if not data_rows:
        print("No se extrajeron datos. Revisa si el ZIP es correcto.")
        return

    print(f"   Datos brutos extraídos: {len(data_rows)} picks/bans.")

    df = pl.DataFrame(data_rows)


    meta_stats = df.group_by("champion").agg([
        pl.col("type").filter(pl.col("type") == "PICK").count().alias("tourney_picks"),
        pl.col("type").filter(pl.col("type") == "BAN").count().alias("tourney_bans"),

        pl.col("win").filter(pl.col("type") == "PICK").mean().fill_null(0).alias("tourney_winrate")
    ])


    meta_stats = meta_stats.with_columns(
        (pl.col("tourney_picks") + pl.col("tourney_bans")).alias("tourney_presence"),
        pl.col("champion").str.to_lowercase().alias("champ_key")
    )

    print(f"   Guardando Meta: {meta_stats.height} campeones.")
    meta_stats.write_parquet(OUTPUT_META_FILE)

    print("\nTOP META (PRESENCIA):")
    print(meta_stats.sort("tourney_presence", descending=True).head(5))


process_tournament_data()